In [ ]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt

import helper

## as a function of $\gamma$

In [ ]:
import matplotlib.gridspec as gridspec

def plot_likelihoods(data, sigma_reward_model, N=500, t_step=2):
    gamma_eval_points = tfd.Uniform([-.2,-.2],[1.2,1.2]).sample(N)
    subplots = [plt.subplot(grid_loc) for grid_loc in gridspec.GridSpec(1, 8)]
    for i,sp in enumerate(subplots):
        t=t_step*i+1
        llhs = np.array([helper.model_llh(data['z'][:t], data['r'][:t], gamma=gamma_eval_points[j], sigma_reward=sigma_reward_model, method='np') for j in range(len(gamma_eval_points))])
        sp.scatter(*np.array(gamma_eval_points).T,c=llhs)
        sp.set_aspect('equal')
        sp.set_title('t = '+str(t_step*i+1))
    plt.gcf().set_figheight(20)
    plt.gcf().set_figwidth(20)
    plt.tight_layout()

In [ ]:
alpha_gt = 45
sigma_reward_gt = 0.01
T = 50

data = helper.generate_data(T, alpha=alpha_gt, sigma_reward=sigma_reward_gt)
helper.plot_data(data)

In [ ]:
plot_likelihoods(data, sigma_reward_model=0.4,N=500)

In [ ]:
plot_likelihoods(data, sigma_reward_model=1.,N=2000)

In [ ]:
alpha_gt = 30
sigma_reward_gt = 0.1
sigma_reward_model = 0.3
t_step = 5
T = 60
N_eval_points = 2000

data = helper.generate_data(T, alpha=alpha_gt, sigma_reward=sigma_reward_gt)
plot_likelihoods(helper.generate_data(T, alpha=alpha_gt, sigma_reward=sigma_reward_gt), sigma_reward_model=sigma_reward_model,N=N_eval_points,t_step=t_step)

In [ ]:
t = 1
gamma_eval_points = tfd.Uniform([-.2,-.2],[1.2,1.2]).sample(2000)
llhs = np.array([helper.model_llh(data['z'][:t], data['r'][:t], gamma=gamma_eval_points[j], sigma_reward=sigma_reward_model) for j in range(len(gamma_eval_points))])
plt.scatter(*np.array(gamma_eval_points).T,c=llhs)
plt.gca().set_aspect('equal')
plt.gcf().set_figheight(2.5)
plt.gcf().set_figwidth(2.5)

## Analytic solution

In [ ]:
t = 2

from scipy.stats import multivariate_normal
x = np.linspace(-.25, 1.25, 40)
y = np.linspace(-.25, 1.25, 40)
X, Y = np.meshgrid(x, y)
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X; pos[:, :, 1] = Y

mu, sigma = helper.gamma_posterior_analytic(zs=data['z'][:t], rs=data['r'][:t], sigma_r=sigma_reward_model, Sigma_0=100*np.eye(2))
Z = multivariate_normal.pdf(pos, mean = mu, cov = sigma)
plt.contourf(X, Y, Z, 20)
plt.colorbar();

In [ ]:
t_step = 4
sigma_reward_model = .2
subplots = [plt.subplot(grid_loc) for grid_loc in gridspec.GridSpec(1, 8)]
for j,sp in enumerate(subplots):
    x = np.linspace(-.25, 1.25, 40)
    y = np.linspace(-.25, 1.25, 40)
    X, Y = np.meshgrid(x, y)
    pos = np.empty(X.shape + (2,))
    pos[:, :, 0] = X; pos[:, :, 1] = Y
    t = t_step*j+1
    mu, sigma = helper.gamma_posterior_analytic(zs=data['z'][:t], rs=data['r'][:t], sigma_r=sigma_reward_model, Sigma_0=100*np.eye(2))
    Z = multivariate_normal.pdf(pos, mean = mu, cov = sigma)
    sp.contourf(X, Y, Z, 20)
    sp.set_aspect('equal')
    sp.set_title('t = '+str(t_step*j+1))
plt.gcf().set_figheight(20)
plt.gcf().set_figwidth(20)
plt.tight_layout()

## as a function of $\alpha$

In [ ]:
alpha_gt = 90
for i in range(20):
    data = helper.generate_data(5,alpha=alpha_gt,sigma_reward=0.001)
    alphas = np.arange(0, 360, 5)
    p_gamma = np.array([helper.model_llh_by_alpha(data['z'], data['r'], alpha=alpha, sigma_reward=1) for alpha in alphas])
    plt.plot(alphas,p_gamma)
    plt.axvline(x=alpha_gt)


In [ ]:
alpha_gt = 90
data = helper.generate_data(5,alpha=alpha_gt,sigma_reward=0.001)

for t in range(10):
    alphas = np.arange(0, 360, 5)
    p_gamma = np.array([helper.model_llh_by_alpha(data['z'][:t+1], data['r'][:t+1], alpha=alpha, sigma_reward=1) for alpha in alphas])
    plt.plot(alphas,p_gamma)
    plt.axvline(x=alpha_gt)
